In [1]:
import os
from datetime import datetime
import numpy as np
import data_preparation as sf
import rnn_enc_dec as red


#PROJECT_ROOT_DIR = "/home/ubuntu/TroubledLife"
PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
MODEL_CHECKPOINTS_DIR = os.path.join(PROJECT_ROOT_DIR, "model_checkpoints")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)

In [2]:
#sf.generate_life_policy_data(no_of_policies=10000, runtime=5, trouble=False, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

#sf.generate_life_policy_data(no_of_policies=2000, runtime=5, trouble=False, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_train = \
    sf.load_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_test = \
    sf.load_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_length_train, max_policy_history_length_train = \
    sf.get_policy_history_lengths(policy_histories=policy_histories_train)

policy_histories_length_test, max_policy_history_length_test = \
    sf.get_policy_history_lengths(policy_histories=policy_histories_test)

max_policy_history_length = max(max_policy_history_length_train, max_policy_history_length_test)

In [3]:
# Pad the histories up to maximum length of both, train and test set

# policy_histories_train = \
#     sf.pad_life_policy_histories(policy_histories=policy_histories_train,
#                                           policy_histories_lengths=policy_histories_length_train,
#                                           max_policy_history_length=max_policy_history_length)
# 
# policy_histories_test = \
#     sf.pad_life_policy_histories(policy_histories=policy_histories_test,
#                                           policy_histories_lengths=policy_histories_length_test,
#                                           max_policy_history_length=max_policy_history_length)
# 
# # Save padded data, since always generating and padding takes too long
# policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
# policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

# Extract features and labels from dataset as numpy.ndarray(s)
train_labels, train_features, train_seq_lengths =\
    sf.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_policy_history_length, binary_classification=True)
test_labels, test_features, test_seq_lengths =\
    sf.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_policy_history_length, binary_classification=True)

train_data = sf.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

In [12]:
import tensorflow as tf


tf.reset_default_graph()
tf.set_random_seed(42)

# Our hyperparameters
n_steps = max_policy_history_length
n_inputs = 2
n_layers = 3
n_neurons = 200
n_latent = 100
learning_rate = 0.0001
bidirectional = True

# Placeholders for our input sequences
seq_length = tf.placeholder(tf.int32, [None], name="seq_length")
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")

latent_vector = red.encoder(X=X, seq_length=seq_length, n_layers=n_layers, n_neurons=n_neurons, n_latent=n_latent)

decoder_sequence_test = red.decoder(n_latent=n_latent, n_layers=n_layers, n_neurons=n_neurons, n_outputs=n_inputs, 
                               latent_vector=latent_vector, X=X, seq_length=seq_length, training=True, reuse=False)

tvars = tf.trainable_variables()

for var in tvars:
    print(var) 
    
loss_test1 = tf.reduce_mean(tf.reduce_mean(tf.square(decoder_sequence_test - X), axis=2), axis=1)
loss_test2 = tf.reduce_mean(tf.square(decoder_sequence_test - X))
trouble_pred = (loss_test1 > 1000)

# Both are needed when using precision and recall metrics
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

saver = tf.train.Saver(var_list=tvars)

encoder states shape: (?, 600)
latent_vector shape: (?, 100)
state_input shape: (?, 200)
decoder initial state shape:  (?, 600)


decoder outputs rnn_output shape: (?, ?, 2)
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_0/basic_rnn_cell/kernel:0' shape=(202, 200) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_0/basic_rnn_cell/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_1/basic_rnn_cell/kernel:0' shape=(400, 200) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_1/basic_rnn_cell/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_2/basic_rnn_cell/kernel:0' shape=(400, 200) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_2/basic_rnn_cell/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'P_Encoder_2_Latent/W:0' shape=(600, 100) dtype=float32_ref>
<tf.Variable 'P_Encoder_2_Latent/b:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'Latent_2_P_Decoder/W:0' shape=(100, 200) dtype=float32_ref>
<tf.Variable 'Latent_2_P_Decoder/b:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'P_Deco

In [13]:
from time import time

start = time()

with tf.Session() as sess:
    init_g.run()
    init_l.run()

    saver.restore(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "rnn_enc_dec.ckpt"))    
    
    trPred, lTest1, lTest2 = sess.run([trouble_pred, loss_test1, loss_test2], feed_dict={X: test_features, seq_length: test_seq_lengths})

    #test_labels = np.concatenate((test_labels, trPred), axis=0)
    
elapsed = time() - start
print("Time to finish %d", elapsed)


INFO:tensorflow:Restoring parameters from /Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/model_checkpoints/rnn_enc_dec.ckpt


Time to finish %d 0.9253368377685547


In [14]:
print (lTest2)
print (lTest1)
print(trPred)


2.31491e+06
[ 4653699.5    1909077.625  3728246.75  ...,  1981770.125  3948016.75
  1261135.125]
[ True  True  True ...,  True  True  True]
